In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import math
from scipy.optimize import curve_fit

In [2]:
Data = pd.read_excel("Infiltration_example.xlsx")
Data.tail()########Data.tail()

,date,treatment,plot,pressure,suction,time,volume
571,2021-05-03,8,4,7,-7 cm,300,64.0
572,2021-05-03,8,4,7,-7 cm,600,50.5
573,2021-05-03,8,4,7,-7 cm,900,37.5
574,2021-05-03,8,4,7,-7 cm,1200,24.5
575,2021-05-03,8,4,7,-7 cm,1500,12.0


# Note
Convert to date format to datetime and check the data types

In [3]:
Data.dtypes

date         datetime64[ns]
treatment             int64
plot                  int64
pressure              int64
suction              object
time                  int64
volume              float64
dtype: object

In [4]:
dates = [datetime(2021,3,27), datetime(2021,5,3)]

# Note
calculate Square Root of time(s)

In [5]:
Data["sqrt"] = np.sqrt(Data["time"])
Data

,date,treatment,plot,pressure,suction,time,volume,sqrt
0,2021-03-27,2,1,1,-1 cm,0,49.1,0.000000
1,2021-03-27,2,1,1,-1 cm,300,45.8,17.320508
2,2021-03-27,2,1,1,-1 cm,600,42.4,24.494897
3,2021-03-27,2,1,1,-1 cm,900,38.9,30.000000
4,2021-03-27,2,1,1,-1 cm,1200,34.8,34.641016
...,...,...,...,...,...,...,...,...
571,2021-05-03,8,4,7,-7 cm,300,64.0,17.320508
572,2021-05-03,8,4,7,-7 cm,600,50.5,24.494897
573,2021-05-03,8,4,7,-7 cm,900,37.5,30.000000
574,2021-05-03,8,4,7,-7 cm,1200,24.5,34.641016


# Note
Import Parameter table (read excel table)

In [6]:
Para = pd.read_excel("Parameter.xlsx")
Para.head()

,soil type,alpha,n&ho,-0.5,-1,-2,-3,-4,-5,-6,-7
0,sand,0.145,2.68,2.835701,2.404070,1.727908,1.241921,0.892621,0.641565,0.461120,0.331427
1,loamy sand,0.124,2.28,2.985300,2.786831,2.428600,2.116417,1.844363,1.607280,1.400674,1.220625
2,sandy loam,0.075,1.89,3.877062,3.887982,3.909913,3.931969,3.954148,3.976453,3.998884,4.021441
3,loam,0.036,1.56,5.461148,5.717657,6.267384,6.869965,7.530482,8.254505,9.048139,9.918077
4,silt,0.016,1.37,7.921451,8.177401,8.714378,9.286617,9.896433,10.546292,11.238825,11.976835


# Note
use 'for ... in ...' to calculate each date, pressure, treatment, plot   
use 'ie = condition' to define conditions   
use '.loc conditons' to look for the values according to the conditions  
use 'def func...return...curve_fit...' to make curvefit and get parameter   
use '.loc conditons...value...' to select values according to the condition   
use 'dateframe.append conditons...'to append dataframe according to conditions  

those are old version 

In [7]:
# create an empty DataFrame with the desired column names
df = pd.DataFrame(columns=['treatment','plot',"pressure",'K'])

# iterate through the loops
for i in [2, 4, 6, 8]:
    for j in [1, 2, 3, 4]:
        for k in [1, 5, 7]:  
            for t in dates:#########'27/3/2021', '3/5/2021', '25/5/2021', '2/6/2021', '24/6/2021'
                ie = (Data['treatment'] == i) & (Data['plot'] == j) & (Data['pressure'] == k) & (Data['date'] == t)
                i0 = ie & (Data['time'] == 0)#####(Data['treatment'] == i) & (Data['plot'] == j) & (Data['pressure'] == k)
                Data.loc[ie, "infiltration"] = (Data.loc[i0, 'volume'].values[0]- Data[ie]["volume"]) / (math.pi * np.square(2.25))
                Data[ie]
                def func(x, a, b):
                    return a*x**2 - b*x ### np.pow(x,2)
                xdata =  Data[ie]["sqrt"]
                ydata = Data[ie]["infiltration"]
                popt, pcov = curve_fit(func, xdata, ydata)
                C1 = popt[0]
                type(C1)
                suction = -k
                ir = (Para["soil type"] == "sandy loam")#######type(Para[ie]["alpha"])
                Para.loc[ir, 'alpha'].values[0]
                a = Para.loc[ir, 'alpha'].values[0]
                Para.loc[ir, 'n&ho'].values[0]
                n = Para.loc[ir, 'n&ho'].values[0]
                aa = 2.25 * a
                nn = 2.92 if (n > 1.9) else 7.5
                A = (11.65 * (np.power(n,0.1) - 1) * math.exp(nn * (n - 1.9) * a * suction)) / (np.power(aa,0.91)) ##2.25 is the radius from 0.5cm - 6cm.
                K = C1 / A * 10000
                
                # create a new DataFrame with the current iteration's data
                new_data = {'treatment': i, 'plot': j, 'pressure': k, 'K': K, 'date':t}
                
                # concatenate the new DataFrame with the existing one
                df = pd.concat([df, pd.DataFrame(new_data, index=[0])], ignore_index=True)


In [8]:
df

,treatment,plot,pressure,K,date
0,2,1,1,2.095519,2021-03-27
1,2,1,1,11.206997,2021-05-03
2,2,1,5,0.858634,2021-03-27
3,2,1,5,7.096933,2021-05-03
4,2,1,7,0.475703,2021-03-27
...,...,...,...,...,...
91,8,4,1,11.291056,2021-05-03
92,8,4,5,2.111968,2021-03-27
93,8,4,5,8.540455,2021-05-03
94,8,4,7,1.655170,2021-03-27


In [ ]:
df.to_excel('K.xlsx', index=False)
df

,treatment,plot,pressure,K,date
0,2,1,1,2.095519,2021-03-27
1,2,1,1,11.206997,2021-05-03
2,2,1,5,0.858634,2021-03-27
3,2,1,5,7.096933,2021-05-03
4,2,1,7,0.475703,2021-03-27
...,...,...,...,...,...
91,8,4,1,11.291056,2021-05-03
92,8,4,5,2.111968,2021-03-27
93,8,4,5,8.540455,2021-05-03
94,8,4,7,1.655170,2021-03-27
